<a href="https://colab.research.google.com/github/SOL1archive/KoGrammar/blob/main/test-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  import google.colab
  IN_COLAB = True
  
except:
  IN_COLAB = False

IN_COLAB

True

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install torchmetrics
!pip install --upgrade accelerate
!pip install rouge
!pip install accelerate
!pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://test.pypi.org/simple/, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import timeit
import datetime
import os
import gc
from collections import namedtuple
from pprint import pprint
from tqdm import tqdm

import numpy as np
import pandas as pd

import tensorboard
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR, CyclicLR
import torchmetrics
from torchsummary import summary

from datasets import load_dataset, load_from_disk, concatenate_datasets, DatasetDict, Dataset
from transformers import get_linear_schedule_with_warmup
from transformers import DataCollatorForSeq2Seq
from transformers import BartConfig, T5Config
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
tokenized_dataset_path = 'drive/MyDrive/projects/KoGrammar/data/gogamza-kobart-base-v2_tokenized_dataset'
baseline_checkpoint_path = 'drive/MyDrive/projects/KoGrammar/models/230510-16_32'
distil_checkpoint_path = 'drive/MyDrive/projects/KoGrammar/models/small_model_230517-19_30'
tiny_distil_checkpoint_path = 'drive/MyDrive/projects/KoGrammar/models/distil_tiny_230518-06_28'
quantized_distil_checkpoint_path = 'drive/MyDrive/projects/KoGrammar/models/small_model_230517-19_30-quantized'
quantized_tiny_distil_checkpoint_path = 'drive/MyDrive/projects/KoGrammar/models/distil_tiny_230518-06_28-quantized'

In [ ]:
baseline_config = BartConfig.from_json_file(baseline_checkpoint_path + '/config.json')
distil_config = BartConfig.from_json_file(distil_checkpoint_path + '/config.json')
tiny_distil_config = BartConfig.from_json_file(tiny_distil_checkpoint_path + '/config.json')
quantized_distil_config = BartConfig.from_json_file(quantized_distil_checkpoint_path + '/config.json')
quantized_tiny_distil_config = BartConfig.from_json_file(quantized_tiny_distil_checkpoint_path + '/config.json')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
if os.path.exists(tokenized_dataset_path):
    tokenized_dataset = load_from_disk(tokenized_dataset_path)
    print(tokenized_dataset)
else:
    print("Tokenized dataset not found")

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1016426
    })
    train_baseline: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 508213
    })
    train_distil: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 508212
    })
    valid: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 56468
    })
    test: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 56469
    })
})


## Baseline Test

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('drive/MyDrive/projects/KoGrammar/models/230510-16_32')

baseline_model = AutoModelForSeq2SeqLM.from_pretrained(baseline_checkpoint_path).to(device)
distil_model = AutoModelForSeq2SeqLM.from_pretrained(distil_checkpoint_path).to(device)
tiny_distil_model = AutoModelForSeq2SeqLM.from_pretrained(tiny_distil_checkpoint_path).to(device)

_ = baseline_model.eval(), distil_model.eval(), tiny_distil_model.eval()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at drive/MyDrive/projects/KoGrammar/models/small_model_230517-19_30 and are newly initialized: ['model.decoder.layers.3.self_attn.q_proj.weight', 'model.decoder.layers.5.self_attn.q_proj.bias', 'model.decoder.layers.3.encoder_attn.q_proj.weight', 'model.decoder.layers.3.fc2.weight', 'model.decoder.layers.5.final_layer_norm.bias', 'model.decoder.layers.3.encoder_attn.v_proj.weight', 'model.decoder.layers.3.encoder_attn_layer_norm.weight', 'model.decoder.layers.4.final_layer_norm.weight', 'model.decoder.layers.3.encoder_attn.out_proj.bias', 'model.decoder.layers.3.encoder_attn.v_proj.bias', 'model.d

In [ ]:
quantized_distil_model = AutoModelForSeq2SeqLM.from_pretrained(quantized_distil_checkpoint_path, load_in_8bit=True, device_map='auto')
quantized_tiny_distil_model = AutoModelForSeq2SeqLM.from_pretrained(quantized_tiny_distil_checkpoint_path, load_in_8bit=True, device_map='auto')

_ = quantized_distil_model.eval(), quantized_tiny_distil_model.eval()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.


In [ ]:
testset = tokenized_dataset['test'].shuffle()
test_sample = testset.shuffle().select(range(1))
test_sample

Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1
})

In [ ]:
def generate_seq(model, tokenizer, input):
    generated_ids = model.generate(**input)
    generated_text = tokenizer.decode(generated_ids.squeeze(0), skip_special_tokens=True)
    
    return generated_text

def generate_input_target(model, tokenizer, input, label):
    input_text = tokenizer.decode(input['input_ids'].squeeze(0), skip_special_tokens=True)
    generated_text = generate_seq(model, tokenizer, input)
    target_text = tokenizer.decode(label.squeeze(0), skip_special_tokens=True)
    
    return {
        'input_text': input_text,
        'generated_text': generated_text, 
        'target_text': target_text
    }

def generate_from_data(model, tokenizer, data):
    label = data['labels']
    input_data = dict()
    input_data['input_ids'] = data['input_ids']
    input_data['attention_mask'] = data['attention_mask']

    return generate_input_target(model, tokenizer, input_data, label)

def eval(model, tokenizer, input_seq, label, metric, options = dict()):
    generated_input_target = generate_input_target(model, tokenizer, input_seq, label)
    score = metric.compute(
        generated_input_target['generated_text'], 
        generated_input_target['target_text'],
        **options
    )

    return score

In [ ]:
testset = tokenized_dataset['test'].with_format("torch", device=device)
test_sample = testset.shuffle().select(range(1))
output = generate_from_data(baseline_model, tokenizer, test_sample)

input_data = dict()
input_data['input_ids'] = test_sample['input_ids']
input_data['attention_mask'] = test_sample['attention_mask']
distil_output = generate_seq(distil_model, tokenizer, input_data)
tiny_distil_output = generate_seq(tiny_distil_model, tokenizer, input_data)
quantized_distil_output = generate_seq(quantized_distil_model, tokenizer, input_data)
quantized_tiny_distil_output = generate_seq(quantized_tiny_distil_model, tokenizer, input_data)

print(output['input_text'], 
      'baseline: ' + output['generated_text'], 
      'distil: ' + distil_output,
      'tiny_distil: ' + tiny_distil_output,
      'quantized_distil: ' + quantized_distil_output,
      'quantized_tiny_distil: ' + quantized_tiny_distil_output,
      output['target_text'], 
      sep='\n\n'
      )

ᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏ

baseline: ᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏ

distil: ᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏ

tiny_distil: ᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏ

quantized_distil: 

quantized_tiny_distil: 

ᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏ


In [ ]:
bleu = evaluate.load('google_bleu')
rouge = Rouge()
result_df_dict = dict()
model_lt = [
    ('distil', distil_model), 
    ('tiny_distil', tiny_distil_model), 
    ('baseline', baseline_model), 
    #('quantized_distil', quantized_distil_model), 
    #('quantized_tiny_distil', quantized_tiny_distil_model),
]

performance_dict = dict()

for name, model in model_lt:
    accuracy_lt = []
    bleu_score_lt = []
    rouge_score_lt = []
    time_score_lt = []

    test_sample = testset.shuffle().select(range(2000))
    total_time = 0
    for example in tqdm(test_sample):
        data = dict()
        for key in example:
            data[key] = example[key].unsqueeze(0)
        start = timeit.default_timer()
        output = generate_from_data(model, tokenizer, data)
        end = timeit.default_timer()
        generated_text = output['generated_text']
        target_text = output['target_text']

        try:
            #accuracy_score = accuracy.compute(predictions=generated_text, references=target_text, tokenizer=tokenizer)
            #bleu_score = bleu.compute(predictions=generated_text, references=target_text, tokenizer=tokenizer)
            bleu_score = sentence_bleu([target_text], generated_text, smoothing_function=SmoothingFunction().method1)
            rouge_score = rouge.get_scores(generated_text, target_text)[0]['rouge-2']['f']
            #rouge_score = rouge.compute(predictions=generated_text, references=target_text)
        except ValueError:
            continue
        
        total_time += end - start
        #accuracy_lt.append(accuracy_score)
        bleu_score_lt.append(bleu_score)
        rouge_score_lt.append(rouge_score)
        time_score_lt.append(end - start)

    bleu_score_series = pd.Series(bleu_score_lt)
    rouge_score_series = pd.Series(rouge_score_lt)
    time_score_series = pd.Series(time_score_lt)
    print('BLEU: ' + str(bleu_score_series.mean()), 
          'ROUGE: ' + str(rouge_score_series.mean()), 
          'Time: ' + str(total_time), 
          'Mean Time: ' + str(time_score_series.mean()), 
          sep='\t'
          )
    result_df = pd.concat([bleu_score_series, rouge_score_series, time_score_series], axis=1)
    result_df.columns = ['BLEU-2', 'ROUGE-2 F1', 'Mean Time']
    result_df_dict[name] = result_df
    performance_dict[name] = total_time

100%|██████████| 2000/2000 [05:58<00:00,  5.57it/s]


BLEU: 0.7788133735457531	ROUGE: 0.5502721033933605	Time: 343.32936048699867	Mean Time: 0.17552625791768847


100%|██████████| 2000/2000 [05:46<00:00,  5.77it/s]


BLEU: 0.7854816860026788	ROUGE: 0.5425506501322703	Time: 331.4697660039874	Mean Time: 0.16946307055418577


100%|██████████| 2000/2000 [05:37<00:00,  5.92it/s]

BLEU: 0.8022218305572155	ROUGE: 0.564976884227559	Time: 323.1128355939827	Mean Time: 0.1651062011210949


In [ ]:
for name, model in model_lt:
    num = sum(p.numel() for p in model.model.parameters())

    print(name + ':', num, sep='\t')

distil:	123859968
tiny_distil:	123859968
baseline:	123859968


In [ ]:
result_df_dict['distil']

In [ ]:
result_df_dict['baseline']

In [ ]:
result_df_dict.keys()

In [ ]:
for name, result_df in result_df_dict.items():
    print(name)
    result_df.to_json(f'drive/MyDrive/projects/KoGrammar/result-data/{name}-test-result.json')

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5), ncols=3, nrows=1)
for i, (name, result_df) in enumerate(result_df_dict.items()):
    sns.kdeplot(data=result_df, 
                fill=True, 
                palette="crest", 
                common_grid=True, 
                cut=1, 
                ax=ax[i],
                )
    
    ax[i].set_title(name)